#                       Перевірка розподілу мандатів на місцевих виборах

На сайті ЦВК знаходимо посилання на сторінку з результатами виборів до районної/міської/обласної ради.

Зразок лінку - http://www.cvk.gov.ua/pls/vm2015/PVM057?PID112=12&PID102=884&PF7691=884&PT001F01=100&rej=0&pt00_t001f01=100

In [16]:
# Вставляємо лінк сюди
url = 'http://www.cvk.gov.ua/pls/vm2015/PVM057?PID112=12&PID102=884&PF7691=884&PT001F01=100&rej=0&pt00_t001f01=100' 

In [17]:
import pandas as pd
# Імпортуємо таблицю з результатами з сайту ЦВК
df = pd.read_html(url)[3]
df = df.drop(df.index[0])
df.columns = ['party','votes','mandates','perc']
df[['votes','mandates']] = df[['votes','mandates']].apply(pd.to_numeric)
# Підраховуємо відсоток голосів кожної партії
df['perc'] = df.votes / df.votes.sum() * 100
# Визначаємо партії що подолали 5-ти відсотковий бар'єр та підраховуємо виборчу квоту.
df = df[df.perc>=5]
df1 = pd.read_html(url)[2]
df1.columns = ['raz','dva']
kilk_dep = df1.dva.iloc[0]
print('Загальна кількість мандатів у раді - ' +str(kilk_dep))
kvota = df.votes.sum() / kilk_dep
print('Виборча квота - ' + str(kvota))
df['mandates_kvota'] = df.votes/kvota
# Кількість мандатів за розподілена за цілими числами
df['mand_round']= round(df.mandates_kvota-0.5,0)
# Підраховуємо дробові залишки
df['chastka'] = df.mandates_kvota - df.mand_round
# Визначаємо кількість нерозподілених мандатів
rizn_mand = kilk_dep - df.mand_round.sum()
df = df.sort_values('chastka',ascending=False)
# Фінальний розподіл без дробових залишків
df['mandates_final'] = df.mand_round + 0
df = df.reset_index(drop=True)
# Розподіл мандатів за дробовими залишками
for i in range(0,int(rizn_mand)):
    df.at[i, 'mandates_final'] = df.mand_round.iloc[i] + 1
# Різниця між кількістю мандатів між даними ЦВК та нашим підрахунком
df['riznytsia'] = df['mandates'] -df['mandates_final']
# Перевірка на помилки
ac = df[df['riznytsia' ] !=0]
if len(ac.index)> 0:
    print('Вкрадені мандати')
    # Визначаємо хто отримав та втратив мандат
    df1 = ac[ac.riznytsia>0]
    dovj = len(df1.index)
    parties = []
    while dovj > 0:
        parties.append(df1.party.iloc[dovj-1])
        dovj -= 1
    print('Партії що отримали зайві мандати -' + str(parties))
    df1 = ac[ac.riznytsia<0]
    dovj = len(df1.index)
    partiesl = []
    while dovj > 0:
        partiesl.append(df1.party.iloc[dovj-1])
        dovj -= 1
    print('Партії що втратили мандати - ' + str(partiesl))
else:
    print('Помилок не виявлено') 

Загальна кількість мандатів у раді - 84
Виборча квота - 7124.238095238095
Вкрадені мандати
Партії що отримали зайві мандати -['Політична Партія "Опозиційний блок"', 'Політична партія "Об’єднання "САМОПОМІЧ"']
Партії що втратили мандати - ['політична партія Всеукраїнське об’єднання "Батьківщина"', 'ПОЛІТИЧНА ПАРТІЯ "УКРАЇНСЬКЕ ОБ’ЄДНАННЯ ПАТРІОТІВ – УКРОП"']


Таблиця розподілу мандатів

In [18]:
df.sort_values('mandates_final',ascending=False)

,party,votes,mandates,perc,mandates_kvota,mand_round,chastka,mandates_final,riznytsia
0,"ПАРТІЯ ""БЛОК ПЕТРА ПОРОШЕНКА ""СОЛІДАРНІСТЬ""",156378,22,22.732929,21.950137,21.0,0.950137,22.0,0.0
4,"політична партія Всеукраїнське об’єднання ""Бат...",119619,16,17.389212,16.790427,16.0,0.790427,17.0,-1.0
1,"Політична партія ""Об’єднання ""САМОПОМІЧ""",63365,10,9.211475,8.894284,8.0,0.894284,9.0,1.0
2,"Політична партія ""Радикальна партія Олега Ляшк...",63311,9,9.203625,8.886705,8.0,0.886705,9.0,0.0
3,"ПОЛІТИЧНА ПАРТІЯ ""УКРАЇНСЬКЕ ОБ’ЄДНАННЯ ПАТРІО...",55644,7,8.089060,7.810519,7.0,0.810519,8.0,-1.0
6,"Політична партія ""Наш край""",51042,7,7.420060,7.164556,7.0,0.164556,7.0,0.0
7,"політична партія Всеукраїнське об’єднання ""Сво...",50689,7,7.368744,7.115006,7.0,0.115006,7.0,0.0
5,"Політична Партія ""Опозиційний блок""",38388,6,5.580527,5.388366,5.0,0.388366,5.0,1.0
